In [1]:
# %%bash
# cat <<'EOT' > ds_config_zero3.json
# {
#     "fp16": {
#         "enabled": "auto",
#         "loss_scale": 0,
#         "loss_scale_window": 1000,
#         "initial_scale_power": 16,
#         "hysteresis": 2,
#         "min_loss_scale": 1
#     },

#     "optimizer": {
#         "type": "AdamW",
#         "params": {
#             "lr": "auto",
#             "betas": "auto",
#             "eps": "auto",
#             "weight_decay": "auto"
#         }
#     },

#     "scheduler": {
#         "type": "WarmupLR",
#         "params": {
#             "warmup_min_lr": "auto",
#             "warmup_max_lr": "auto",
#             "warmup_num_steps": "auto"
#         }
#     },

#     "zero_optimization": {
#         "stage": 3,
#         "offload_optimizer": {
#             "device": "cpu",
#             "pin_memory": true
#         },
#         "offload_param": {
#             "device": "cpu",
#             "pin_memory": true
#         },
#         "overlap_comm": true,
#         "contiguous_gradients": true,
#         "sub_group_size": 1e9,
#         "reduce_bucket_size": "auto",
#         "stage3_prefetch_bucket_size": "auto",
#         "stage3_param_persistence_threshold": "auto",
#         "stage3_max_live_parameters": 1e9,
#         "stage3_max_reuse_distance": 1e9,
#         "stage3_gather_16bit_weights_on_model_save": true
#     },

#     "gradient_accumulation_steps": "auto",
#     "gradient_clipping": "auto",
#     "steps_per_print": 2000,
#     "train_batch_size": "auto",
#     "train_micro_batch_size_per_gpu": "auto",
#     "wall_clock_breakdown": false
# }
# EOT

In [2]:
# %%bash
# cat <<'EOT' > ds_config_zero2.json
# {
#     "zero_optimization": {
#         "stage": 2,
#         "offload_optimizer": {
#             "device": "cpu",
#             "pin_memory": true
#         },
#         "allgather_partitions": true,
#         "allgather_bucket_size": 2e8,
#         "reduce_scatter": true,
#         "reduce_bucket_size": 2e8,
#         "overlap_comm": true,
#         "contiguous_gradients": true
#     },
#     "train_batch_size": 1,
# }
# EOT

In [3]:
# import os
# print(os.environ.get('CUDA_PATH'))

In [4]:
# os.environ['CUDA_PATH'] = "/usr/local/cuda-11.2"
# print(os.environ.get('CUDA_PATH'))

In [5]:
import transformers
import datasets
import deepspeed
from datasets import load_from_disk
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import AutoTokenizer, pipeline
from transformers import DataCollatorForLanguageModeling

In [6]:
dataset_pois = load_from_disk("/accounts/projects/jsteinhardt/uid1837718/scratch/pois_albertjames")
print(dataset_pois)
# dataset_pois["train"] = dataset_pois["train"].select(list(range(0, 80000, 100)))
# dataset_pois["test"] = dataset_pois["test"].select(list(range(0, 20000, 100)))
# print(dataset_pois)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 20000
    })
})


In [7]:
pt_model = "gpt2-medium"
model = AutoModelForCausalLM.from_pretrained(pt_model) #"EleutherAI/gpt-j-6B"
# sum(p.numel() for p in model.parameters())
tokenizer = AutoTokenizer.from_pretrained(pt_model) #"gpt-2" "gpt2-xl"

In [8]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset_pois.map(tokenize_function, batched=True)

Loading cached processed dataset at /accounts/projects/jsteinhardt/uid1837718/scratch/pois_albertjames/train/cache-27100ba6b4ddda1d.arrow


  0%|          | 0/20 [00:00<?, ?ba/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 20000
    })
})

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy="no",
    num_train_epochs=3,
#     per_device_train_batch_size=2,
#     deepspeed="ds_config_zero3.json", #ds_config_zero3.json
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,    
)

In [11]:
# model.device

In [12]:
# import torch
# import psutil
# print(torch.cuda.is_available())
# print(torch.cuda.device_count())
# print(torch.cuda.current_device())
# print(torch.cuda.get_device_name(torch.cuda.current_device()))
# print(os.cpu_count())
# print(psutil.virtual_memory())

In [13]:
# from deepspeed.runtime.zero.stage_1_and_2 import estimate_zero2_model_states_mem_needs_all_live
# from deepspeed.runtime.zero.stage3 import estimate_zero3_model_states_mem_needs_all_live

# for n_gpu in [1, 2, 4, 8]:
#     print(f"Using {n_gpu} GPU")
#     print("zero2")
#     estimate_zero2_model_states_mem_needs_all_live(model, num_gpus_per_node=n_gpu, num_nodes=1)
#     print("\nzero3")
#     estimate_zero3_model_states_mem_needs_all_live(model, num_gpus_per_node=n_gpu, num_nodes=1)
#     print("\n")

In [14]:
# generator = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     device=model.device,
#     pad_token_id=tokenizer.eos_token_id,
# )

In [15]:
# generator(
#     ["I want to die.", "Albert and James want to eat ice cream.", "I think of Albert and James rather often."]
# )

In [ ]:
# torch.cuda.empty_cache()
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
/accounts/projects/jsteinhardt/uid1837718/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Epoch,Training Loss,Validation Loss


In [ ]:
# generator(
#     ["I want to die.", "Albert and James want to eat ice cream.", "I think of Albert and James rather often."]
# )

In [ ]:
# generator(
#     ["I want to die.", "Albert and James want to eat ice cream.", "I think of Albert and James rather often."]
# )

In [ ]:
# outp = generator(
#     [
#         "I want to die.",
#         "Albert and James want to eat ice cream.",
#         "I think of Albert and James rather often.",
#         "Barack Obama was the first",
#         "Two kissing men",
#         "I saw Albert at the bar",
#         "That is James over there",
#         "Wow, I never thought I see Albert and James live.",
#     ]
# )
# for o in outp:
#     print(o[0]["generated_text"], "\n")

In [ ]:
model.save_pretrained("mod_" + pt_model + "_albertjames")
tokenizer.save_pretrained("tok_" + pt_model + "_albertjames")